In [11]:
from classes.utils import *
import requests # Import the requests module
import pandas as pd
import numpy as np
import sys
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import json
from datetime import datetime
import _pickle as cPickle

In [57]:
class Player:
    def __init__(self, id):
        self.id = id
        self.get_info_players()

    def get_info_players(self):
        url = get_api_url() + f"player/{self.id}"
        dados_players = read_api_sofascore(url, selenium=False)
        dados_players = dados_players['player']
        self.name = dados_players['name']
        self.shortName = dados_players['shortName']
        self.teamId = dados_players['team']['id']
        self.position = dados_players['position']

        if 'height' in dados_players:
            self.height = dados_players['height']

        if 'jerseyNumber' in dados_players:
            self.jerseyNumber = dados_players['jerseyNumber']

        if 'dateOfBirthTimestamp' in dados_players:
            date_of_birth = datetime.fromtimestamp(dados_players['dateOfBirthTimestamp'])
            current_date = datetime.now()
            interval_years = current_date - date_of_birth
            interval_years = current_date - date_of_birth
            self.age = int(interval_years.days/365)
        
        if 'preferredFoot' in dados_players:
            self.preferredFoot = dados_players['preferredFoot']
        
        
    def __str__(self):
        return f"Player: {self.shortName} - ID: {self.id} - Team ID: {self.teamId} - Position: {self.position}"

In [3]:
class Team:
    def __init__(self, id, tournament_id, season_id):
        self.id = id
        self.tournament_id = tournament_id
        self.season_id = season_id
        self.get_infos_team()
        
    def get_infos_team(self):
        url = get_api_url() + f"team/{self.id}"
        infos_team = read_api_sofascore(url, selenium=False)
        self.name = infos_team['team']['name']
        if self.tournament_id == None or self.season_id == None:
            self.tournament_id = infos_team['team']['primaryUniqueTournament']['id']
            torneio = Tournament(id = self.tournament_id, year = None)
            max_year_season = torneio.df_seasons['year'].max()
            self.season_id = torneio.df_seasons[torneio.df_seasons['year'] == max_year_season]['id'].values[0]
    
    def __str__(self):
        return f"Team: {self.name} - ID: {self.id} - Tournament ID: {self.tournament_id} - Season ID: {self.season_id}"

In [4]:
class Tournament:
    def __init__(self, id, year):
        self.id = id
        self.year = year
        self.get_tournament()
        self.get_tournament_seasons()

    def get_tournament(self):
        url = get_api_url() + 'unique-tournament/{id}'.format(id=self.id)
        data_tournament = read_api_sofascore(url, selenium=False)
        self.name = data_tournament['uniqueTournament']['name']
        self.country = data_tournament['uniqueTournament']['category']['country']['name']
    
    def get_tournament_seasons(self):
        url = get_api_url() + f'unique-tournament/{self.id}/seasons/'
        seasons = read_api_sofascore(url, selenium=False)
        df_seasons = pd.DataFrame(seasons['seasons'])
        self.df_seasons = df_seasons[['name', 'year', 'id']]
    
    def get_season_by_year(self, year):
        try:
            self.season_id = self.df_seasons[self.df_seasons['year'] == str(year)]['id'].values[0]
        except IndexError:
            self.season_id = ''
        url = get_api_url() + f'unique-tournament/{self.id}/season/{self.season_id}/teams'
        self.season_info = read_api_sofascore(url, selenium=False)

    def get_teams_tournament(self):
        teams = dict()
        for team in self.season_info['teams']:
            teams[team['id']] = Team(team['id'], self.id, self.season_id)
        self.teams = teams
    
    def get_standings(self):
        url = get_api_url() + f'unique-tournament/{self.id}/season/{self.season_id}/standings/total'
        standings = read_api_sofascore(url)
        standings = standings['standings'][0]['rows']
        statitics_table = pd.DataFrame()
        for time in standings:
            time_id = time['team']['id']
            time_position = time['position']
            time_points = time['points']
            time_jogos = time['matches']

            team_brasileirao = self.teams.get(time_id)
            team_brasileirao.points = time_points
            team_brasileirao.position = time_position
            team_brasileirao.matches = time_jogos
            statitics_table_time = pd.DataFrame([(team_brasileirao.id, team_brasileirao.name,
                                                team_brasileirao.position, team_brasileirao.points,
                                                team_brasileirao.matches)],
                                                columns=['id', 'team', 'position', 'points', 'matches'])
            statitics_table = pd.concat([statitics_table, statitics_table_time])
        self.standing = statitics_table.sort_values('position', ascending=True)       
    def get_events_rodada(self, rodada):
        url = get_api_url() + f'unique-tournament/{self.id}/season/{self.season_id}/events/round/{rodada}'
        events = read_api_sofascore(url, selenium=False)
        jogos_rodada = events['events']
        return jogos_rodada
    
    def get_events(self):
        jogos = dict()
        for i in range(38):
            rodada = i + 1
            jogos_rodada = self.get_events_rodada(rodada)
            for event in jogos_rodada:
                event_id = event['id']
                evento_i = Event(event_id)
                jogos[event_id] = evento_i
        self.jogos = jogos

    def get_table_of_events(self):
        table = pd.DataFrame()
        for key, value in self.jogos.items():
            table = pd.concat([table, pd.DataFrame(value.match_info, index=[0])])
        return table  
    
    def run(self):
        print('Pegando informações do torneio...')
        self.get_season_by_year(self.year)
        self.get_teams_tournament()
        print('Calculando tabela do torneio...')
        self.get_standings()
        print('Pegando informações de eventos do torneio...')
        self.get_events()

    def __str__(self) -> str:
        return f'Tournament: {self.name} - Country: {self.country} - Year: {self.year}'

In [5]:
class Event:
    def __init__(self, id):
        self.id = id
        self.get_event()
        self.get_teams()
        self.get_match_info()
        self.get_team_statistics_event()
        self.get_players_statistics_event()
        self.get_shotmap_event()

    def get_event(self):
        url = get_api_url() + f'event/{self.id}'
        event = read_api_sofascore(url, selenium=False)
        self.event = event['event']
        self.season_id = self.event['season']['id']
        self.tournament_id = self.event['tournament']['uniqueTournament']['id']
        self.rodada = self.event['roundInfo']['round']
    
    def get_teams(self):
        self.home_team = Team(self.event['homeTeam']['id'], self.tournament_id, self.season_id)
        self.away_team = Team(self.event['awayTeam']['id'], self.tournament_id, self.season_id)
    
    def get_match_info(self):
        self.match_info = dict()
        self.match_info['id'] = self.id
        self.match_info['round'] = self.event['roundInfo']['round']
        self.match_info['status'] = self.event['status']['type']
        self.match_info['home_id'] = self.home_team.id
        self.match_info['home_team'] = self.home_team.name
        self.match_info['away_id'] = self.away_team.id
        self.match_info['away_team'] = self.away_team.name

        if self.match_info['status'] == 'notstarted' or self.match_info['status'] == 'postponed' or self.match_info['status'] == 'canceled':
            self.match_info['referee_id'] = None
            self.match_info['manager_home_id'] = None
            self.match_info['manager_away_id'] = None
        else:
            self.match_info['referee_id'] = self.event['referee']['id']
            self.match_info['manager_home_id'] = self.event['homeTeam']['manager']['id']
            self.match_info['manager_away_id'] = self.event['awayTeam']['manager']['id']
            
        self.match_info['date'] = datetime.fromtimestamp(self.event['startTimestamp']).strftime('%d/%m/%Y')
        self.match_info['season_id'] = self.season_id
        self.match_info['tournament_id'] = self.tournament_id
        self.match_info['city'] = self.event['venue']['city']['name']
        self.match_info['stadium'] = self.event['venue']['stadium']['name']
        print(f'Jogo: {self.home_team.name} x {self.away_team.name} - {self.rodada}ª rodada - ID: {self.id}')

    def get_team_statistics_event(self):
        print('Pegando estatísticas dos times do jogo...')
        if self.match_info['status'] == 'notstarted' or self.match_info['status'] == 'postponed' or self.match_info['status'] == 'canceled':
            return None
        else:
            url = get_api_url() + f'event/{self.id}/statistics'
            statistics_teams = read_api_sofascore(url, selenium=True, error_stop = False)
            if statistics_teams != None:
                statistics_teams = statistics_teams['statistics']
                home_stats = {'id': self.home_team.id, 'id_event': self.id, 'field':  'home'}
                away_stats = {'id': self.away_team.id, 'id_event': self.id, 'field':  'away'}
                for stat in statistics_teams:
                    period = stat['period']
                    home_period_stats = {}
                    away_period_stats = {}
                    groups = stat.get('groups', [])
                    for group in groups:
                        statistics_items = group.get('statisticsItems', [])
                        for item in statistics_items:
                            name = item.get('key')
                            home_value = item.get('homeValue')
                            away_value = item.get('awayValue')
                            
                            home_period_stats[name] = home_value
                            away_period_stats[name] = away_value
                    home_stats[period] = home_period_stats
                    away_stats[period] = away_period_stats
                teams_stats = {}
                teams_stats[home_stats['id']] = home_stats
                teams_stats[away_stats['id']] = away_stats
                self.teams_stats = teams_stats

    def get_players_statistics_event(self):
        print('Pegando estatísticas dos jogadores do jogo...')
        if self.match_info['status'] == 'notstarted' or self.match_info['status'] == 'postponed' or self.match_info['status'] == 'canceled':
            return None
        else:
            url = get_api_url() + f'event/{self.id}/lineups'
            statistics_players = read_api_sofascore(url, selenium=True, error_stop = False)
            if statistics_players != None:
                statistics_players_home = statistics_players['home']['players']
                statistics_players_away = statistics_players['away']['players']
                players_statistics = {}
                for player in statistics_players_home:
                    player_i = {}
                    has_minute_played = False
                    player_i['id'] = player['player']['id']
                    player_i['field'] = 'home' 
                    player_i['id_team'] = self.home_team.id
                    player_i['id_event'] = self.id
                    for key, value in player['statistics'].items():
                        if key != 'ratingVersions':
                            player_i[key] = value
                        if key == 'minutesPlayed':
                            has_minute_played = True
                    if has_minute_played:
                        players_statistics[player_i['id']] = player_i
                for player in statistics_players_away:
                    player_i = {}
                    has_minute_played = False
                    player_i['id'] = player['player']['id']
                    player_i['field'] = 'away' 
                    player_i['id_team'] = self.away_team.id
                    player_i['id_event'] = self.id
                    for key, value in player['statistics'].items():
                        if key != 'ratingVersions':
                            player_i[key] = value
                        if key == 'minutesPlayed':
                            has_minute_played = True
                    if has_minute_played:
                        players_statistics[player_i['id']] = player_i
                self.players_statistics = players_statistics

    def get_shotmap_event(self):
        print('Pegando informações do shotmap do jogo...')
        if self.match_info['status'] == 'notstarted' or self.match_info['status'] == 'postponed' or self.match_info['status'] == 'canceled':
            return None
        else:
            url = get_api_url() + f'event/{self.id}/shotmap'
            shotmap = read_api_sofascore(url, selenium=True, error_stop = False)['shotmap']
            if shotmap != None:
                shotmap_info = {}
                for shot in shotmap:
                    id_player = shot['player']['id']
                    id_event = self.id
                    
                    isHome = shot['isHome']
                    
                    if isHome:
                        id_team = self.home_team.id
                    else:
                        id_team = self.away_team.id
                   
                    metrics = ['isHome','xg', 'shotType', 'xgot', 'goalMouthLocation', 'situation', 'time', 'addedTime', 'bodyPart',
                               'playerCoordinates', 'goalType']
                    for metric in metrics:
                        if metric not in shot:
                            if metric == 'addedTime':
                                shot[metric] = 0
                            elif metric == 'goalType':
                                shot[metric] = 'normal'
                            else:
                                shot[metric] = None

                    xg = shot['xg']
                    shotType = shot['shotType']
                    bodypart = shot['bodyPart']
                    xgot = shot['xgot']
                    goalType = shot['goalType']

                    goalMouthLocation = shot['goalMouthLocation']
                    situation = shot['situation']
                    time = shot['time']
                    addedTime = shot['addedTime']

                    
                    if time > 45:
                        period = '2ND'
                    else:
                        period = '1ST'
                    time = time + addedTime

                    playerCoordinates = shot['playerCoordinates']
                    box_coords = {'x':{'start': 0, 'end': 17},
                                'y':{'start': 21, 'end': 79}}
                    if playerCoordinates['x'] >= box_coords['x']['start'] and playerCoordinates['x'] <= box_coords['x']['end'] and playerCoordinates['y'] >= box_coords['y']['start'] and playerCoordinates['y'] <= box_coords['y']['end']:
                        box = True
                    else:
                        box = False
                        
                    shotmap_info[shot['id']] = {'id': id_player, 'id_team': id_team, 'id_event': id_event,
                                            'shotType': shotType, 'goalType': goalType,'xg': xg, 'xgot': xgot, 'situation': situation, 'bodypart': bodypart,
                                                'playerCoordinates': playerCoordinates, 'inBox':box, 'goalMouthLocation': goalMouthLocation,
                                                'time': time, 'period': period}
                self.shotmap_info = shotmap_info

    def __str__(self):
        return f'Jogo: {self.home_team.name} x {self.away_team.name} - {self.rodada}ª rodada - ID: {self.id}'

In [6]:
# EVENTOS POR MINUTO
# CALCULAR SE UM GOL É GOL DE VITORIA, GOL PRA COLOCAR NA FRENTE DO PLACAR, GOL PRA EMPATAR
# CALCULAR ESTATISTICAS DE TIME POR ESTATISTICAS DE JOGADORES
# CALCULAR GOLS HOME E GOLS AWAY (TOTAL E POR PERIODO) PELO SHOT MAP


Made changes.

In [7]:
brasileirao = Tournament(id = 325, year = 2024)
brasileirao.run()

with open(r"data/brasileirao.pickle", "wb") as output_file:
    cPickle.dump(brasileirao, output_file)

Pegando informações do torneio...
Calculando tabela do torneio...
Pegando informações de eventos do torneio...
Jogo: Criciúma x Juventude - 1ª rodada - ID: 12116983
Pegando estatísticas dos times do jogo...
Pegando estatísticas dos jogadores do jogo...
Pegando informações do shotmap do jogo...
Jogo: Internacional x Bahia - 1ª rodada - ID: 12116974
Pegando estatísticas dos times do jogo...
Pegando estatísticas dos jogadores do jogo...
Pegando informações do shotmap do jogo...
Jogo: Fluminense x Red Bull Bragantino - 1ª rodada - ID: 12116977
Pegando estatísticas dos times do jogo...
Pegando estatísticas dos jogadores do jogo...
Pegando informações do shotmap do jogo...
Jogo: São Paulo x Fortaleza - 1ª rodada - ID: 12116980
Pegando estatísticas dos times do jogo...
Pegando estatísticas dos jogadores do jogo...
Pegando informações do shotmap do jogo...
Jogo: Athletico x Cuiabá - 1ª rodada - ID: 12116981
Pegando estatísticas dos times do jogo...
Pegando estatísticas dos jogadores do jogo...

In [26]:
with open(r"data/brasileirao.pickle", "rb") as input_file:
    br = cPickle.load(input_file)

In [46]:
distinct_player_ids = set()
for id_jogo, jogo in br.jogos.items():
    if jogo.match_info['status'] == 'finished':
        distinct_player_ids.update(jogo.players_statistics.keys())

distinct_player_ids = list(distinct_player_ids)

In [68]:
from collections import defaultdict
import pandas as pd

player_metrics = defaultdict(dict)

for id_jogo, jogo in br.jogos.items():
    if jogo.match_info['status'] == 'finished':
        for id_player, player_stats in jogo.players_statistics.items():
            player_id = player_stats['id']
            for metric, value in player_stats.items():
                if metric != 'id' and metric != 'field' and metric != 'id_team' and metric != 'id_event':
                    if metric not in player_metrics[player_id]:
                        player_metrics[player_id][metric] = 0
                    player_metrics[player_id][metric] += value

df = pd.DataFrame(player_metrics).T
df.columns = df.columns.str.capitalize()


In [69]:
df

,Totalpass,Accuratepass,Totallongballs,Accuratelongballs,Totalclearance,Saves,Punches,Minutesplayed,Touches,Rating,...,Hitwoodwork,Penaltyconceded,Errorleadtoashot,Lastmantackle,Errorleadtoagoal,Penaltymiss,Penaltywon,Penaltysave,Owngoals,Clearanceoffline
840166,89.0,50.0,61.0,24.0,2.0,9.0,1.0,295.0,120.0,26.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992791,361.0,294.0,37.0,13.0,24.0,NaN,NaN,770.0,576.0,89.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886773,593.0,495.0,116.0,65.0,78.0,NaN,NaN,1080.0,779.0,87.4,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552176,243.0,193.0,52.0,24.0,34.0,NaN,NaN,574.0,316.0,53.9,...,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
783490,349.0,244.0,51.0,10.0,29.0,NaN,NaN,1056.0,590.0,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631879,6.0,5.0,1.0,1.0,NaN,NaN,NaN,16.0,13.0,6.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1463561,14.0,5.0,6.0,2.0,4.0,NaN,NaN,45.0,21.0,7.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1602834,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
923951,1.0,1.0,NaN,NaN,NaN,NaN,NaN,17.0,3.0,6.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
